In [355]:
import pandas as pd
import numpy as np

### Read and preprocess

In [488]:
visits = pd.read_csv('http://rocker-data-engineering-task.storage.googleapis.com/data/visits.csv')
#use datetime
visits = visits.drop("Unnamed: 0",axis=1)
visits['timestamp'] = pd.to_datetime(visits['timestamp'], unit='s')


In [478]:
customers = pd.read_json('http://rocker-data-engineering-task.storage.googleapis.com/data/customers.json', lines=True)

In [479]:
#list of all data files, this would be easier to pick if directly from a GCS bucket
loan_csv_list = 'http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2017-10.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2017-11.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2017-12.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-1.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-10.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-11.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-12.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-2.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-3.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-4.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-5.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-6.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-7.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-8.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-9.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-1.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-10.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-2.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-3.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-4.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-5.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-6.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-7.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-8.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-9.csv'
loan_csv_list = loan_csv_list.split(', ')
#for url in loan_csv_list:
#    print(url)

In [498]:
#combine multiple dataframes and use datetime instead of epoch
dfs = [pd.read_csv(url) for url in loan_csv_list]
loan = pd.concat(dfs, ignore_index=True)
loan['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

##format webvisit_id
loan['webvisit_id'] = loan['webvisit_id'].str.replace(r'[()]', '').str.replace(r',', '')
#webvisit_id object to numeric
loan['webvisit_id'] = pd.to_numeric(loan['webvisit_id'], errors='coerce').convert_dtypes() 
loan['webvisit_id'] =loan['webvisit_id'].fillna(0)
loan = loan.drop("Unnamed: 0",axis=1)


loan = loan.replace(0, np.nan)

In [499]:
loan.dtypes
#loan.isnull().sum()

id                       int64
user_id                  int64
timestamp       datetime64[ns]
loan_amount              int64
loan_purpose            object
outcome                 object
interest               float64
webvisit_id              Int64
dtype: object

In [500]:
customers.dtypes

id           int64
name        object
ssn         object
birthday    object
gender      object
city        object
zip_code    object
dtype: object

In [501]:
visits.dtypes

id                        int64
timestamp        datetime64[ns]
referrer                 object
campaign_name            object
dtype: object

### lets look at the data

In [502]:
loan.head()

,id,user_id,timestamp,loan_amount,loan_purpose,outcome,interest,webvisit_id
0,9546870,33593540,2017-10-28 03:42:29,80000,Buying a pet,ACCEPTED,8.7,<NA>
1,36469880,41352735,2017-10-24 23:35:34,270000,Refinance existing loans,REJECTED,6.7,72572646
2,22554526,70055796,2017-10-03 20:59:49,240000,Home purchase,ACCEPTED,7.0,54404642
3,27906548,17577009,2017-10-22 08:23:47,190000,Buying a pet,REJECTED,14.0,<NA>
4,46601919,15988868,2017-10-21 11:53:43,20000,Home purchase,REJECTED,14.0,<NA>


In [503]:
customers.head()

,id,name,ssn,birthday,gender,city,zip_code
0,20427847,Μάριος Γαρουφαλής,172-10-3586,1978-05-09,F,Γρεβενά,ΤΚ 763 78
1,74075652,Μόσχα Διαμαντοπούλου,140-57-5668,1994-03-05,F,Κομοτηνή,75665
2,31170608,Κλυταιμνήστρα Πέτση,548-79-9954,1971-08-24,M,Φλώρινα,ΤΚ 49056
3,39640871,Ευγενία Στρατογιάννη,036-43-6966,1988-12-19,M,Φλώρινα,65934
4,60718455,Χαρίτος-Τίμων Τσατσάνης,467-48-0823,1989-07-21,F,Θεσσαλονίκη,ΤΚ 38335


In [504]:
visits.head()
#dfa =visits.pivot_table(index=['id'], aggfunc='size')

,id,timestamp,referrer,campaign_name
0,80139861,2018-08-07 22:30:13,Twitter,display1
1,80139861,2017-11-11 03:32:59,Facebook,display1
2,32212575,2018-04-11 03:23:08,Google,display2
3,32212575,2017-12-03 20:00:18,Facebook,display3
4,32212575,2018-09-23 20:23:40,Google,display2


### merge all csv files

In [509]:
#data = pd.merge(loan, customers, how='left', left_on='user_id', right_on='id')
#loan.isnull().sum()

#dropna() is used as a workaround to exclude left join on nan values
data1 = pd.merge(loan[pd.notnull(loan.webvisit_id)], visits, how='left', left_on='webvisit_id', right_on='id',  suffixes=('_loan', '_visits'))
data1.sort_values('user_id')
#foo.merge(bar[pd.notnull(bar.id)], how='left', on='id')

,id_loan,user_id,timestamp_loan,loan_amount,loan_purpose,outcome,interest,webvisit_id,id_visits,timestamp_visits,referrer,campaign_name
859,82367130,46037,2017-11-07 07:33:39,90000,Refinance existing loans,ACCEPTED,6.7,54354971,54354971.0,2019-06-30 04:35:18,Facebook,display2
858,82367130,46037,2017-11-07 07:33:39,90000,Refinance existing loans,ACCEPTED,6.7,54354971,54354971.0,2019-01-19 07:51:44,Google,display2
857,82367130,46037,2017-11-07 07:33:39,90000,Refinance existing loans,ACCEPTED,6.7,54354971,54354971.0,2018-11-25 01:07:37,Google,display3
856,82367130,46037,2017-11-07 07:33:39,90000,Refinance existing loans,ACCEPTED,6.7,54354971,54354971.0,2018-12-18 14:27:07,Facebook,display3
2857,10408516,49747,2018-12-19 01:06:35,220000,Buying a pet,REJECTED,6.7,3833218,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2936,12536361,99981350,2018-12-29 09:28:22,230000,Buying a pet,REJECTED,7.0,19779542,19779542.0,2018-08-14 04:04:54,Google,display1
2935,12536361,99981350,2018-12-29 09:28:22,230000,Buying a pet,REJECTED,7.0,19779542,19779542.0,2017-12-15 08:54:43,Twitter,display1
2933,12536361,99981350,2018-12-29 09:28:22,230000,Buying a pet,REJECTED,7.0,19779542,19779542.0,2017-12-16 05:34:28,Twitter,display1
2937,12536361,99981350,2018-12-29 09:28:22,230000,Buying a pet,REJECTED,7.0,19779542,19779542.0,2018-06-09 20:40:22,Facebook,display3


In [510]:
data2 = pd.merge(loan, customers, how='left', left_on='user_id', right_on='id', suffixes=('_loan', '_customers'))
data2.sort_values('user_id')

,id_loan,user_id,timestamp,loan_amount,loan_purpose,outcome,interest,webvisit_id,id_customers,name,ssn,birthday,gender,city,zip_code
6336,89754689,29817,2018-05-06 20:42:40,290000,Home purchase,ACCEPTED,8.7,<NA>,29817,Νεοκλής Γιαννακέας,539-03-6064,1995-08-10,M,Φλώρινα,17331
1143,82367130,46037,2017-11-07 07:33:39,90000,Refinance existing loans,ACCEPTED,6.7,54354971,46037,Ελισσαίος Κουτσικόπουλος,219-88-6096,1996-08-05,M,Κιλκίς,ΤΚ 464 00
3773,10408516,49747,2018-12-19 01:06:35,220000,Buying a pet,REJECTED,6.7,3833218,49747,Ευθαλία-Λεμονιά Τυμβίου,350-67-3292,1995-02-13,F,Καρπενήσι,ΤΚ 592 82
3772,73276428,49747,2018-12-15 14:46:43,170000,Refinance existing loans,ACCEPTED,14.0,<NA>,49747,Ευθαλία-Λεμονιά Τυμβίου,350-67-3292,1995-02-13,F,Καρπενήσι,ΤΚ 592 82
9847,30014830,55228,2019-02-16 21:20:10,230000,buying a pet,ACCEPTED,6.7,<NA>,55228,Ηλίας Αραμπατζής,423-19-0023,1971-10-28,M,Λευκάδα,ΤΚ 896 53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4918,23949707,99952010,2018-03-30 16:53:45,210000,Refinance existing loans,ACCEPTED,6.7,69058961,99952010,Μάρκος Μπλανάς,840-29-7553,1984-10-27,F,Κέρκυρα,87807
3858,12536361,99981350,2018-12-29 09:28:22,230000,Buying a pet,REJECTED,7.0,19779542,99981350,Αυγουστίνα Κεσίση,264-79-1994,1994-02-23,M,Λάρισα,926 42
10838,26695034,99991063,2019-03-03 11:51:39,280000,Refinance existing loans,REJECTED,6.7,<NA>,99991063,Λυγερή Καμπούρη,361-83-7372,1981-01-29,F,Πρέβεζα,ΤΚ 714 18
12230,72693506,99992386,2019-06-05 05:57:31,60000,Refinance existing loans,ACCEPTED,6.7,<NA>,99992386,Σμαράγδα Οικονομοπούλου,785-02-0513,1970-01-13,F,Μεσολόγγι,ΤΚ 798 62


In [515]:
data3 = pd.merge(data2, data1, how='left', on=['id_loan','user_id', 'loan_amount', 'loan_purpose', 'outcome', 'interest', 'webvisit_id'])
data3.sort_values('user_id')

,id_loan,user_id,timestamp,loan_amount,loan_purpose,outcome,interest,webvisit_id,id_customers,name,ssn,birthday,gender,city,zip_code,timestamp_loan,id_visits,timestamp_visits,referrer,campaign_name
9339,89754689,29817,2018-05-06 20:42:40,290000,Home purchase,ACCEPTED,8.7,<NA>,29817,Νεοκλής Γιαννακέας,539-03-6064,1995-08-10,M,Φλώρινα,17331,NaT,NaN,NaT,NaN,NaN
1667,82367130,46037,2017-11-07 07:33:39,90000,Refinance existing loans,ACCEPTED,6.7,54354971,46037,Ελισσαίος Κουτσικόπουλος,219-88-6096,1996-08-05,M,Κιλκίς,ΤΚ 464 00,2017-11-07 07:33:39,54354971.0,2019-01-19 07:51:44,Google,display2
1665,82367130,46037,2017-11-07 07:33:39,90000,Refinance existing loans,ACCEPTED,6.7,54354971,46037,Ελισσαίος Κουτσικόπουλος,219-88-6096,1996-08-05,M,Κιλκίς,ΤΚ 464 00,2017-11-07 07:33:39,54354971.0,2018-12-18 14:27:07,Facebook,display3
1666,82367130,46037,2017-11-07 07:33:39,90000,Refinance existing loans,ACCEPTED,6.7,54354971,46037,Ελισσαίος Κουτσικόπουλος,219-88-6096,1996-08-05,M,Κιλκίς,ΤΚ 464 00,2017-11-07 07:33:39,54354971.0,2018-11-25 01:07:37,Google,display3
1668,82367130,46037,2017-11-07 07:33:39,90000,Refinance existing loans,ACCEPTED,6.7,54354971,46037,Ελισσαίος Κουτσικόπουλος,219-88-6096,1996-08-05,M,Κιλκίς,ΤΚ 464 00,2017-11-07 07:33:39,54354971.0,2019-06-30 04:35:18,Facebook,display2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5657,12536361,99981350,2018-12-29 09:28:22,230000,Buying a pet,REJECTED,7.0,19779542,99981350,Αυγουστίνα Κεσίση,264-79-1994,1994-02-23,M,Λάρισα,926 42,2018-12-29 09:28:22,19779542.0,2019-05-27 04:55:28,Twitter,display1
5656,12536361,99981350,2018-12-29 09:28:22,230000,Buying a pet,REJECTED,7.0,19779542,99981350,Αυγουστίνα Κεσίση,264-79-1994,1994-02-23,M,Λάρισα,926 42,2018-12-29 09:28:22,19779542.0,2017-12-16 05:34:28,Twitter,display1
16146,26695034,99991063,2019-03-03 11:51:39,280000,Refinance existing loans,REJECTED,6.7,<NA>,99991063,Λυγερή Καμπούρη,361-83-7372,1981-01-29,F,Πρέβεζα,ΤΚ 714 18,NaT,NaN,NaT,NaN,NaN
18195,72693506,99992386,2019-06-05 05:57:31,60000,Refinance existing loans,ACCEPTED,6.7,<NA>,99992386,Σμαράγδα Οικονομοπούλου,785-02-0513,1970-01-13,F,Μεσολόγγι,ΤΚ 798 62,NaT,NaN,NaT,NaN,NaN


### Save the output

In [516]:
data3.to_csv('data_foobank.csv')